In [ ]:
import os
import json
import pandas as pd
from pymongo import MongoClient
from pymongo.server_api import ServerApi
import re
from collections import Counter

In [ ]:
uri = ""

In [ ]:
# Create a new client and connect to the server
client = MongoClient(uri, server_api=ServerApi('1'))
# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

In [ ]:
# Adjusting the code to correctly handle the structure of the currency_prices data

price_data = []
for merchant in data['prices']:
    merchant_name = merchant['merchant']
    for price_info in merchant['prices']:
        for currency_entry in price_info['currency_prices']:
            for date_key, price in currency_entry.items():
                if 'dateSeen' in date_key:
                    dates = price
                    currency_number = date_key.split('_')[1]
                    price_key = f'currency_{currency_number}'
                    amount = float(currency_entry[price_key]['$numberDouble'])
                    for date in dates:
                        price_data.append((merchant_name, datetime.fromisoformat(date.replace('Z', '')), amount))

# Sort data by date
price_data.sort(key=lambda x: x[1])

# Separate data by merchants for plotting
merchants = set([x[0] for x in price_data])
plt.figure(figsize=(10, 6))

for merchant in merchants:
    merchant_data = [(x[1], x[2]) for x in price_data if x[0] == merchant]
    dates, prices = zip(*merchant_data)
    plt.plot(dates, prices, label=merchant)

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Price History by Merchant')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()